In [2]:
#
import base58
import bech32
import binascii
import ecdsa
import hashlib
import json
import os
import subprocess
import time

def dSHA256(data):
    hash_1 = hashlib.sha256(data).digest()
    hash_2 = hashlib.sha256(hash_1).digest()
    return hash_2

def hash160(s):
    '''sha256 followed by ripemd160'''
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def privkey_to_pubkey(privkey):
    signing_key = ecdsa.SigningKey.from_string(privkey, curve=ecdsa.SECP256k1) # Don't forget to specify the curve
    verifying_key = signing_key.get_verifying_key()

    # Use this code block if the address you gave corresponds to the compressed public key
    x_cor = bytes.fromhex(verifying_key.to_string().hex())[:32] # The first 32 bytes are the x coordinate
    y_cor = bytes.fromhex(verifying_key.to_string().hex())[32:] # The last 32 bytes are the y coordinate
    if int.from_bytes(y_cor, byteorder="big", signed=True) % 2 == 0: # We need to turn the y_cor into a number.
        public_key = bytes.fromhex("02" + x_cor.hex())
    else:
        public_key = bytes.fromhex("03" + x_cor.hex())
    return public_key

In [3]:
# Functions related to generating bitcoin addresses

def encode_base58(s):
    BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    count = 0
    for c in s:  
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes(s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:  
        num, mod = divmod(num, 58)
        result = BASE58_ALPHABET[mod] + result
    return prefix + result

def encode_base58_checksum(b):
    return encode_base58(b + dSHA256(b)[:4])

# https://en.bitcoin.it/wiki/Wallet_import_format
def privkey_to_wif(privkey, compressed_pubkey, testnet):
    if testnet:
        prefix = b"\xEF"
    else:
        prefix = b"\x80"
    
    # if the privkey will correspond to a compressed public key
    if compressed_pubkey: 
        extended = prefix + privkey + b"\x01"
    else:
        extended = prefix + privkey 
        
    extendedchecksum = extended + dSHA256(extended)[:4]
    wif = encode_base58(extendedchecksum)
    
    return wif

# https://learnmeabitcoin.com/guide/wif
def wif_to_privkey(private_key_WIF):
    private_key_full = base58.b58decode(private_key_WIF)
    
    # If the WIF encoding includes the optional "01" byte for compressed privKey,
    # do not include it in the final output.
    if len(private_key_full) == 38:
        private_key = private_key_full[1:-5] 
        print("compressed pubkey")
    else:
        private_key = private_key_full[1:-4] 
        print("not compressed pubkey")
    return private_key


def pk_to_p2pkh(compressed, network):
    '''Returns the address string'''
    pk_hash = hash160(compressed)
    if network == "testnet":
        prefix = b'\x6f'
    elif network == "simnet":
        prefix = b'\x3f'
    elif network == "mainnet":
        prefix = b'\x00'
    else:
        return "Enter the network: tesnet/simnet/mainnet"
    return encode_base58_checksum(prefix + pk_hash)

def pk_to_p2sh_p2wpkh(compressed, network):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    rs_hash = hash160(redeemScript)
    if network == "testnet":
        prefix = b"\xc4"
    elif network == "simnet":
        prefix = b'\x7b'
    elif network == "mainnet":
        prefix = b"\x05"
    else:
        return "Enter the network: tesnet/simnet/mainnet"
    return encode_base58_checksum(prefix + rs_hash)


def pk_to_p2wpkh(compressed, network):
    pk_hash = hash160(compressed)
    redeemScript = bytes.fromhex(f"0014{pk_hash.hex()}")
    spk = binascii.unhexlify(redeemScript.hex())
    version = spk[0] - 0x50 if spk[0] else 0
    program = spk[2:]
    if network == "testnet":
        prefix = 'tb'
    elif network == "simnet":
        prefix = 'sb'
    elif network == "mainnet":
        prefix = 'bc'
    else:
        return "Enter the network: tesnet/simnet/mainnet"
    return bech32.encode(prefix, version, program)
    
    
def wif_to_addresses(wif):
#     wif = "cUy9rC6wteKizfu1fgP2abKUWTkJxjqKp2fba91FkU332CFHo6ix"
    privkey = wif_to_privkey(wif)
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, network = "simnet")
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, network = "simnet")

    print("WIF Private key: " + wif)
    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("Address: " + p2pkh_address)
    print("Address: " + p2sh_p2wpkh_address)
    
def privkey_to_addresses(privkey):
    # privkey = bytes.fromhex("AF933A6C602069F1CBC85990DF087714D7E86DF0D0E48398B7D8953E1F03534A")
    public_key = privkey_to_pubkey(privkey)
    p2pkh_address = pk_to_p2pkh(public_key, network = "simnet")
    p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(public_key, network = "simnet")
    p2wpkh_address = pk_to_p2wpkh(public_key, network = "simnet")

    print("Private key: " + privkey.hex())
    print("Public key: " + public_key.hex())
    print("Public key hash: " + hash160(public_key).hex())
    print("p2pkh_address: " + p2pkh_address)
    print("np2wkh_address: " + p2sh_p2wpkh_address)
    print("p2wpkh_address: " + p2wpkh_address)

# Testing escrow tx created by ln-mpc

Note, things to consider:

btcd simnet addresses have their own format.
can mining reward be payed out to p2wpkh?

There was no minimum relay/mempool fee on btcd simnet.

btcd command "searchrawtransaction" may be useful for future. <br>
e.g.  "btcctl --simnet --rpcuser=kek --rpcpass=kek searchrawtransactions rYETLRYoqXCLT5FiSGMDFK9THqro7RuFrG"
where rYETLRYoqXCLT5FiSGMDFK9THqro7RuFrG is the address. <br>
--addrindex must be set in order to use this command

In [17]:
# Generate example priv/pubkeys
miner_privkey_hex = "1111111111111111111111111111111100000000000000000000000000000000"
miner_pubkey_bytes = privkey_to_pubkey(bytes.fromhex(miner_privkey_hex))
miner_p2wpkh_address = pk_to_p2wpkh(miner_pubkey_bytes, network = "simnet")
miner_p2sh_p2wpkh_address = pk_to_p2sh_p2wpkh(miner_pubkey_bytes, network = "simnet")

#### Stop btcd and start again (from another terminal), with alice's address as the mining address

In [5]:
# Make sure btcd is not already running
subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek stop")

'Post https://localhost:18556: dial tcp [::1]:18556: connect: connection refused'

In [18]:
# start up btcd in simnet mode outside of notebook with this command
str("btcd --txindex --simnet --rpcuser=kek --rpcpass=kek --miningaddr=" + miner_p2sh_p2wpkh_address)

'btcd --txindex --simnet --rpcuser=kek --rpcpass=kek --miningaddr=rYETLRYoqXCLT5FiSGMDFK9THqro7RuFrG'

In [19]:
# generate 1 block and get the utxo
blockhash = json.loads(subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek generate 1"))
block = json.loads(subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek getblock " + blockhash[0]))
print("blockhash: " + str(blockhash[0]))
print("block: " + json.dumps(block, indent=4))

# Assuming no other txs are in the block
mined_txid = block["tx"][0]
print("mined_txid: " + mined_txid)

blockhash: 68f39d710d38f8b62b77fc10d47327cde9da0eb2a90e0546f7c4ce3556e8b6b9
block: {
    "hash": "68f39d710d38f8b62b77fc10d47327cde9da0eb2a90e0546f7c4ce3556e8b6b9",
    "confirmations": 1,
    "strippedsize": 188,
    "size": 188,
    "weight": 752,
    "height": 5663,
    "version": 536870912,
    "versionHex": "20000000",
    "merkleroot": "35bd8a0e899bccaa35b732a0b26fdba3f56f44c6f0543a0a62577d6636d7c211",
    "tx": [
        "35bd8a0e899bccaa35b732a0b26fdba3f56f44c6f0543a0a62577d6636d7c211"
    ],
    "time": 1581592281,
    "nonce": 0,
    "bits": "207fffff",
    "difficulty": 1,
    "previousblockhash": "120bb7d49c3ebc53ee5b1366d2831d4638564ce09b3a5b3d067feb56f32312a5"
}
mined_txid: 35bd8a0e899bccaa35b732a0b26fdba3f56f44c6f0543a0a62577d6636d7c211


#### Decode the coinbase tx (just to view it)

In [20]:
mined_tx = subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek getrawtransaction " + mined_txid)
decoded = subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek decoderawtransaction " + mined_tx)
print(decoded)

{
  "txid": "35bd8a0e899bccaa35b732a0b26fdba3f56f44c6f0543a0a62577d6636d7c211",
  "version": 1,
  "locktime": 0,
  "vin": [
    {
      "coinbase": "021f1608bdc0ca72eb9d957a0b2f503253482f627463642f",
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 50,
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_HASH160 1d2cc47e2a0d77927a333a2165fe2d343b79eefc OP_EQUAL",
        "hex": "a9141d2cc47e2a0d77927a333a2165fe2d343b79eefc87",
        "reqSigs": 1,
        "type": "scripthash",
        "addresses": [
          "rYETLRYoqXCLT5FiSGMDFK9THqro7RuFrG"
        ]
      }
    }
  ]
}


#### Generate 100 more blocks so that the previous coinbase tx can be spent

In [21]:
subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek generate 100");

#### Get the signedEscrowTx from ln-mpc and broadcast to the network

In [26]:
raw_tx = "0200000000010111c2d736667d57620a3a54f0c6446ff5a3db6fb2a032b735aacc9b890e8abd350000000017160014578dd1183845e18d42f90b1a9f3a464675ad2440ffffffff02282300000000000022002023f934ed0743383714a0798d80edb212a192643d1c5cfada3ecd9d5c8e417ab8d8ce052a0100000016001461492b43be394b9e6eeb077f17e73665bbfd455b0247304402201d7128ad2c64ee961badcbaa9d24a011facb3f3a09f6fe2ff36b95cc9d3d4da202206eb51dbfe66374b551fa692ca07010fd17d81bb0b89ba7882fb1f3150d3ea6370121021d5c5194a62b272f98a7d8321e4b3a03add9d61d13a15a00123888b81850cee200000000"

escrow_txid = subprocess.getoutput("btcctl --simnet --rpcuser=kek --rpcpass=kek sendrawtransaction " + raw_tx)

#### EscrowTxid returned by btcd

In [27]:
print(escrow_txid)

-27: TX rejected: transaction already exists
